In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r'C:\Users\i23733\OneDrive - Verisk Analytics\__Projects\buildfax\VeriskLocation_CA_PermitExtract_20200511.csv')

C:\Python38\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (2,7,12,26) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df.dtypes

PermitID                  object
Address1                  object
Address2                  object
City                      object
State                     object
Zip                       object
PermitNum                 object
MasterPermitNum           object
PreferredDate             object
AppliedDate               object
IssuedDate                object
ExpiresDate               object
COIssuedDate              object
CompletedDate             object
HoldDate                 float64
VoidDate                 float64
StatusDate                object
ValuationAmount          float64
Description               object
ProposedUse               object
PermitTypePreferred       object
WorkClass                 object
PermitType                object
PermitTypeDescription     object
PermitClass               object
PermitStatus              object
ProjectName               object
dtype: object

In [4]:
len(df)

3012550

In [ ]:
match = match[['PermitID', 'Description', 'Structure keywords_sim_score',
       'Structure keywords_tags', 'Property/landscaping keywords_sim_score',
       'Property/landscaping keywords_tags', 'Roof keywords_sim_score',
       'Roof keywords_tags', 'General keywords_sim_score',
       'General keywords_tags', 'Company_sim_score', 'Company_tags',
       'Product_sim_score', 'Product_tags', 'Company modified_sim_score',
       'Company modified_tags', 'Product modified_sim_score',
       'Product modified_tags', 'Address1', 'Address2', 'City', 'State', 'Zip','IssuedDate']]

In [8]:
woolsey = pd.read_csv(r'C:\Users\i23733\OneDrive - Verisk Analytics\__Projects\Defensible Space\WoolseyData\final.csv')

In [9]:
woolsey.columns

Index(['fid', 'POINTID', 'OBJECTID', 'DAMAGE', 'Address', 'CITY', 'CALFIREUNI',
       'WHEREFIRES', 'WHATDIDFIR', 'DEFENSIVEA', 'PROPANETAN', 'STRUCTURET',
       'STRUCTUREC', 'NUMBEROFUN', 'NONHABITAB', 'NOOUTBUILD', 'NOOUTBUI_1',
       'ROOFCONSTR', 'EAVES', 'VENTSCREEN', 'EXTERIORSI', 'WINDOWPANE',
       'DECKPORCHO', 'DECKPORCHE', 'PATIOCOVER', 'FENCEATTAC', 'UTILITYMIS',
       'FIRENAME', 'APN', 'ASSESSEDIM', 'YEARBUILT', 'Zip', 'area', 'pct_55',
       'pct_125', 'damage_und', 'pct_damage_buildings', 'pct_damage_damage',
       'withinpcl_125', 'withinpcl_55', '_primary_structure', '_image_id',
       '_capture_date', '_camera_technology', '_gsd', '_parcel_area',
       '_parcel_tree_cover', '_pool_area', '_roof_top_geocode',
       '_ground_elevation', '_detection_iou', '_detection_score',
       '_roof_shape', '_roof_shape_conf', '_roof_material',
       '_roof_material_probs', '_roof_material_conf', '_roof_condition',
       '_roof_condition_probs', '_roof_solar', '_roof_

In [10]:
woolsey = woolsey[['fid','Address','CITY']]

In [11]:
woolsey['Address1'] = woolsey.Address.str.title()

In [31]:
df = df.loc[df.Description.isna() == False]
df_address = df[['PermitID','Address1','City','Zip']]

In [32]:
df_address['partialaddress'] = df_address.Address1.str.title()

<ipython-input-32-5bfbbae97a93>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_address['partialaddress'] = df_address.Address1.str.title()


In [33]:
def getpartialStreet(s):
    s = str(s)
    if len(s)>0:
        x= s.count(' ')
        y = s.split()[0:x]
        z = ' '.join(y)
        return z
    else:
        pass
    
add = '1 Samuel Hanson Ave'

print(getpartialStreet(add))

1 Samuel Hanson


In [34]:
df['PartialStreet'] = df.Address1.apply(lambda x: getpartialStreet(x)).str.title()

In [27]:
woolsey['PartialStreet'] = woolsey.Address.apply(lambda x: getpartialStreet(x)).str.title()

In [35]:
test2 = pd.merge(woolsey, df, how='inner',on='PartialStreet')

In [37]:
len(test2)

11635

In [20]:
test2.to_csv('testmatch2.csv')

In [21]:
import os, warnings
import pandas as pd, numpy as np
from fuzzywuzzy import process, fuzz

C:\Python38\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [38]:
# Make all values lowercase for better matching

big_list = df.apply(lambda x: x.astype(str).str.lower().str.strip())
little_list = woolsey.apply(lambda x: x.astype(str).str.lower().str.strip())

In [39]:
little_list.head()

,fid,Address,CITY,Address1,PartialStreet
0,1,33360 decker school rd,malibu,33360 decker school rd,33360 decker school
1,2,6134 busch dr,malibu,6134 busch dr,6134 busch
2,3,415 westlake blvd,malibu,415 westlake blvd,415 westlake
3,4,34555 pacific coast hwy,malibu,34555 pacific coast hwy,34555 pacific coast
4,5,33457 pacific coast hwy,malibu,33457 pacific coast hwy,33457 pacific coast


In [40]:
# function for the matching. 
def match_address(name, list_addresses, min_score=0):
    # -1 score if no match
    max_score = -1
    # Return empty address for no match
    max_address = ""
    # Iterating over all addresses in the other file
    for address2 in list_addresses:
        address2 = str(address2)
        # Finding fuzzy match score
        score = fuzz.ratio(address, address2)
        # Checking if the match is above the threshold
        if (score > min_score) & (score > max_score):
            max_address = address2
            max_score = score
    return (max_address, max_score)

In [41]:
little_zipcity = little_list.CITY
citylist = list(little_list.CITY)

In [42]:
# a function to remove the duplicates in the list of cities - don't need to do the same city twice
def Remove(duplicate): 
    final_list = [] 
    for num in duplicate: 
        if num not in final_list: 
            final_list.append(num) 
    return final_list

In [43]:
bigcities = list(big_list.City)
bcitylist = Remove(bigcities)

In [44]:
bcitylist.sort()
print(bcitylist)

['acton', 'adelanto', 'agoura', 'agoura hills', 'agua fria', 'aguandulce', 'all cities', 'alpine', 'alta loma', 'altadena', 'anaheim', 'anderson', 'angelus oaks', 'antelope valley', 'apple valley', 'arleta', 'arrow bear lake', 'arrow bear park', 'arrow bear pk', 'arrowbear', 'arrowhead highland', 'arrowhead villas', 'baldwin lake', 'baldy mesa', 'bell canyon', 'bella vista', 'berry creek', 'big bear lake', 'bonita', 'brea', 'brisbane', 'buena park', 'burbank', 'butte valley', 'ca 94515', 'calabasas', 'calistoga', 'canoga park', 'cedar glen', 'cedarpines park', 'cederpines park', 'chatsworth', 'chicago park', 'chico', 'cima', 'claremont', 'clearlake', 'clearlake oaks', 'cloverdale', 'cobb', 'concow', 'cornell', 'cottonwood', 'county of el dorado', 'covina', 'coyote', 'crestline', 'crooked palm rd ventura', 'dart canyon', 'dayton', 'deer lodge park', 'descanso', 'devore', 'downey', 'dulzura', 'duncan', 'durham', 'el cajon', 'el dorado hills', 'el monte', 'el nido', 'encinitas', 'escondid

In [49]:
#citylist = Remove(citylist)
citylist.append('westlake village')
citylist

['malibu',
 'agoura hills',
 'nan',
 'los angeles county',
 'calabasas',
 'westlake village',
 'oak park',
 'canoga park',
 'westlake',
 'los angeles',
 'malibu',
 'malibu beach',
 'westlake village']

In [50]:
biglist = pd.DataFrame(big_list)

In [51]:
def getStreetName(s):
    s = str(s)
    if len(s)>0:
        x= s.count(' ')
        y = s.split()[1:x]
        z = ' '.join(y)
        return z
    else:
        pass

In [52]:
woolsey.Address = woolsey.Address.astype(str)
street_list = []
for a in woolsey.Address:
    if a != np.nan:
        print(a)
        b = getStreetName(a)
        street_list.append(b)
    

33360 Decker School Rd
6134 Busch Dr
415 Westlake Blvd
34555 Pacific Coast Hwy
33457 Pacific Coast Hwy
31839 Lobo Canyon Rd
302 Little Sycamore Canyon Rd
30330 Cornell School Rd
186 N Kanan Dume Rd
33461 Pacific Coast Hwy
5900 Bonsall Dr
5904 Bonsall Dr
540 Westlake Blvd
1521 Decker Canyon Rd
30360 Mulholland Hwy
29951 Harvester Rd
1909 Hazel Nut Ct
33517 Pacific Coast Hwy
29855 Harvester Rd
33269 Mulholland Hwy
33419 Pacific Coast Hwy
31534 Anacapa View Dr
33317 Decker School Rd
30063 Andromeda Ln
5868 Philip Ave
35241 Mulholland Hwy
31823 Lobo Canyon Rd
3648 Decker Canyon Rd
31750 Lobo Canyon Rd
30354 Mulholland Hwy
31536 Anacapa View Dr
30366 Cornell School Rd
1859 Decker School Ln
200 N Kanan Dume Rd
33310 Hassted Dr
422 Westlake Blvd
1921 Hazel Nut Ct
3052 Triunfo Canyon Rd
34685 Pacific Coast Hwy
33257 Decker School Rd
34033 Pacific Coast Hwy
35201 Mulholland Hwy
3554 Decker Canyon Rd
32075 Lobo Canyon Rd
546 Westlake Blvd
33330 Hassted Dr
31900 Lobo Canyon Rd
33505 Pacific Coast

5960 Busch Dr
5703 Calpine Dr
4101 Escondido Dr
129 Fallen Leaf Ln
26124 Roymor Dr
26060 Farmfield Rd
6414 Cavalleri Rd
6485 Zuma View Pl
5334 Ambridge Dr
5374 Chesebro Rd
5855 Belbert Cir
30626 Rayo del Sol Dr
5531 Parkmor Rd
1485 Decker Canyon Rd
6301 Gayton Pl
30622 Morning View Dr
5964 Paseo Canyon Dr
28916 W Beach Ln
28824 Selfridge Dr
2102 Corral Canyon Rd
6254 Porterdale Dr
6403 Sea Star Dr
6000 Trancas Canyon Rd
6418 Trancas Canyon Rd
5822 Ridgebrook Dr
5763 Colodny Dr
7055 Birdview Ave
6717 Portshead Rd
33 Nottingham Rd
7008 Dume Dr
6402 Surfside Dr
26232 Idlewild St
5356 Chesebro Rd
5849 Belbert Cir
30185 Pacific Coast Hwy
4341 Ocean View Dr
5467 Wellesley Dr
28855 Lake Vista Dr
29329 Bluewater Rd
6262 Trancas Canyon Rd
28608 Acacia Glen St
5374 Wellesley Dr
505 Thrift Rd
5900 Busch Dr
5970 Cavalleri Rd
30071 Andromeda Ln
122 Merry Oak Ln
28929 Wight Rd
30183 Pacific Coast Hwy
29855 Westhaven Dr
26038 Redbluff Dr
5436 Wellesley Dr
27017 Esward Dr
28903 Selfridge Dr
5835 Green

26158 Adamor Rd
5585 Wellesley Dr
6851 Fernhill Dr
6968 Dume Dr
6431 Surfside Dr
6419 Surfside Dr
29157 Paiute Dr
5865 Green Meadow Dr
26038 Trana Cir
7061 Birdview Ave
6203 Tapia Dr
1 Sherwood Dr
5528 Colodny Dr
4400 Encinal Canyon Rd
1876 Lookout Rd
6728 Wildlife Rd
30 Nottingham Rd
1561 Lookout Dr
5518 Wellesley Dr
5857 Stonecrest Dr
5661 Colodny Dr
6447 Surfside Dr
6409 Surfside Dr
5844 Woodglen Dr
26914 Helmond Dr
5732 Colodny Dr
26219 Adamor Rd
6123 Tapia Dr
5859 Bonsall Dr
26125 Fairside Rd
24976 Kit Carson Rd
26920 Helmond Dr
26233 Adamor Rd
30272 Belmont Ct
28712 Timberlane St
30545 Pacific Coast Hwy
30602 Vista Sierra Dr
5724 Calpine Dr
1929 Olivera Dr
5555 Fairview Pl
5508 Parkmor Rd
24960 Kit Carson Rd
6133 Paseo Canyon Dr
28950 Boniface Dr
5875 Green Meadow Dr
5539 Fairview Pl
5865 Greenview Rd
6122 Tapia Dr
28859 Boniface Dr
1912 Seminole Dr
5545 Colodny Dr
32700 Vista de Los Ondas St
29335 Heathercliff Rd
27935 Winding Way
26615 Ocean View Dr
6112 Bonsall Dr
6744 Wildlif

In [57]:
street_list

['Decker School',
 'Busch',
 'Westlake',
 'Pacific Coast',
 'Pacific Coast',
 'Lobo Canyon',
 'Little Sycamore Canyon',
 'Cornell School',
 'N Kanan Dume',
 'Pacific Coast',
 'Bonsall',
 'Bonsall',
 'Westlake',
 'Decker Canyon',
 'Mulholland',
 'Harvester',
 'Hazel Nut',
 'Pacific Coast',
 'Harvester',
 'Mulholland',
 'Pacific Coast',
 'Anacapa View',
 'Decker School',
 'Andromeda',
 'Philip',
 'Mulholland',
 'Lobo Canyon',
 'Decker Canyon',
 'Lobo Canyon',
 'Mulholland',
 'Anacapa View',
 'Cornell School',
 'Decker School',
 'N Kanan Dume',
 'Hassted',
 'Westlake',
 'Hazel Nut',
 'Triunfo Canyon',
 'Pacific Coast',
 'Decker School',
 'Pacific Coast',
 'Mulholland',
 'Decker Canyon',
 'Lobo Canyon',
 'Westlake',
 'Hassted',
 'Lobo Canyon',
 'Pacific Coast',
 'Bonsall',
 'Decker School',
 'Deerhead',
 'Westlake',
 'Decker Canyon',
 'Cornell School',
 'N Kanan Dume',
 'Decker School',
 'Yellow Hill',
 'Decker Edison',
 'Hazel Nut',
 'Hazel Nut',
 'Mulholland',
 'Lobo Canyon',
 'Cuthbert'

In [59]:
big_list2 = biglist[biglist['City'].isin(citylist)]
big_list2.Address1 = big_list2.Address1.astype(str)

big_list2['Street'] = big_list2.Address1.apply(lambda x: getStreetName(x)).str.title()
        
bigbig = big_list2[big_list2['Street'].isin(street_list)]
len(bigbig)

C:\Python38\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
<ipython-input-59-bd1beb1a2404>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  big_list2['Street'] = big_list2.Address1.apply(lambda x: getStreetName(x)).str.title()


17388

In [61]:
little_list['Street'] = little_list.Address.apply(lambda x: getStreetName(x)).str.title()

In [63]:
bigbig

,PermitID,Address1,Address2,City,State,Zip,PermitNum,MasterPermitNum,PreferredDate,AppliedDate,...,ProposedUse,PermitTypePreferred,WorkClass,PermitType,PermitTypeDescription,PermitClass,PermitStatus,ProjectName,PartialStreet,Street
22267,29606|ca_thousandoaks,967 triunfo canyon rd,nan,westlake village,ca,91361-1826,16-00002857,nan,2016-07-20,2016-07-20,...,residential planned dev,mech-htg-a/c pmt/general (npc),mechanical work,mr,mech-htg-a/c pmt/general (npc),nan,final inspection complete,nan,967 triunfo canyon,Triunfo Canyon
22304,32522|ca_thousandoaks,967 triunfo canyon rd,nan,westlake village,ca,91361-1826,16-00004771,nan,2016-11-23,2016-11-23,...,residential planned dev,plumbing pmt/general (npc),plumbing work,pr,plumbing pmt/general (npc),nan,final inspection complete,nan,967 triunfo canyon,Triunfo Canyon
22466,71190|ca_thousandoaks,967 triunfo canyon rd,nan,westlake village,ca,91361-1826,10-00002017,nan,2010-07-28,2010-07-28,...,nan,building permit,alteration of single family dwl w/garage,b,building permit,nan,final inspection complete,nan,967 triunfo canyon,Triunfo Canyon
22467,71191|ca_thousandoaks,967 triunfo canyon rd,nan,westlake village,ca,91361-1826,10-00002017,nan,2010-07-28,2010-07-28,...,nan,electrical permit,alteration of single family dwl w/garage,er,electrical permit,nan,final inspection complete,nan,967 triunfo canyon,Triunfo Canyon
22491,72983|ca_thousandoaks,967 triunfo canyon rd,nan,westlake village,ca,91361-1826,10-00003202,nan,2010-11-29,2010-11-29,...,nan,building permit,alteration of single family dwl w/garage,b,building permit,nan,permit printed,nan,967 triunfo canyon,Triunfo Canyon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3012217,992294|ca_losangelescounty,25437 cumberland ln,nan,calabasas,ca,91302-3157,9702060025,nan,1997-02-06,1,...,nan,building permit,residential addition,bl,building permit,nan,permisud,nan,25437 cumberland,Cumberland
3012223,993761|ca_losangelescounty,29829 vista del arroyo,nan,agoura hills,ca,91301-3032,9702210021,nan,1997-02-21,1,...,nan,building permit,retaining wall,bl,building permit,nan,permisud,nan,29829 vista del,Vista Del
3012224,993992|ca_losangelescounty,29829 vista del arroyo,nan,agoura hills,ca,91301-3032,9702240059,nan,1997-02-24,1,...,nan,electrical permit,nan,el,electrical permit,nan,permisud,nan,29829 vista del,Vista Del
3012225,994999|ca_losangelescounty,25437 cumberland ln,nan,calabasas,ca,91302-3157,9703050001,nan,1997-03-05,1,...,nan,combination pool permit,nan,cp,combination pool permit,nan,permisud,nan,25437 cumberland,Cumberland


In [67]:
street_list = Remove(street_list)

In [68]:
len(street_list)

299

In [70]:
# Create matches
new_list = []

for street in street_list:
    # create df for each city to speed up matching
    bigL = bigbig.loc[bigbig['Street'] == street]
    littleL = little_list.loc[little_list['Street'] == street]
    for address in littleL.Address:
        # Find the best match using a set threshold - 75% seems to yield the best results
        print(address)
        address = str(address)
        match = match_address(address, bigL.Address1, 75)
        # New dict for storing data
        newaddress = {}
        newaddress.update({"infile_address" : address})
        newaddress.update({"match_address" : match[0]})
        newaddress.update({"score" : match[1]})
        new_list.append(newaddress)

33360 decker school rd
33317 decker school rd
1859 decker school ln
33257 decker school rd
1849 decker school ln
1900 decker school ln
33317 decker school rd
33317 decker school rd
33165 decker school rd
33169 decker school rd
33169 decker school rd
33170 decker school rd
33058 decker school rd
33058 decker school rd
33370 decker school rd
33160 decker school rd
1714 decker school ln
33208 decker school rd
33208 decker school rd
33052 decker school rd
33052 decker school rd
33064 decker school rd
33064 decker school rd
33046 decker school rd
33046 decker school rd
33046 decker school rd
33050 decker school rd
33210 decker school rd
33206 decker school rd
33271 decker school rd
33054 decker school rd
33056 decker school rd
33060 decker school rd
33200 decker school rd
33315 decker school rd
33261 decker school rd
6134 busch dr
5555 busch dr
5555 busch dr
5617 busch dr
5580 busch dr
5580 busch dr
5841 busch dr
5838 busch dr
5932 busch dr
5936 busch dr
5911 busch dr
5939 busch dr
5927 bus

30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholland hwy
30473 mulholl

32100 triunfo canyon rd
32100 triunfo canyon rd
32100 triunfo canyon rd
32100 triunfo canyon rd
3529 triunfo canyon rd
3529 triunfo canyon rd
3529 triunfo canyon rd
3529 triunfo canyon rd
3529 triunfo canyon rd
3448 triunfo canyon rd
2938 triunfo canyon rd
2938 triunfo canyon rd
2938 triunfo canyon rd
2938 triunfo canyon rd
2938 triunfo canyon rd
2936 triunfo canyon rd
2936 triunfo canyon rd
2936 triunfo canyon rd
3014 triunfo canyon rd
3109 triunfo canyon rd
2854 triunfo canyon rd
2940 triunfo canyon rd
2977 triunfo canyon rd
3074 triunfo canyon rd
3094 triunfo canyon rd
3582 triunfo canyon rd
3061 triunfo canyon rd
3009 triunfo canyon rd
3030 triunfo canyon rd
5874 deerhead rd
5849 deerhead rd
5835 deerhead rd
5859 deerhead rd
5878 deerhead rd
5869 deerhead rd
5842 deerhead rd
5877 deerhead rd
5838 deerhead rd
12460 yellow hill rd
12680 yellow hill rd
12680 yellow hill rd
3650 decker edison rd
400 decker edison rd
3650 decker edison rd
3650 decker edison rd
3960 decker edison rd
2991

3230 cornell rd
3230 cornell rd
4011 cornell rd
4025 cornell rd
3250 cornell rd
2524 sierra creek rd
2534 sierra creek rd
2534 sierra creek rd
2317 sierra creek rd
2305 sierra creek rd
2504 sierra creek rd
2319 waring dr
2361 waring dr
2361 waring dr
2325 waring dr
2305 waring dr
2336 waring dr
2317 waring dr
2333 waring dr
2366 waring dr
2311 waring dr
2345 waring dr
2314 waring dr
30070 triunfo dr
30065 triunfo dr
29840 triunfo dr
29675 triunfo dr
29754 triunfo dr
29704 triunfo dr
30045 triunfo dr
29704 triunfo dr
30005 triunfo dr
30075 triunfo dr
30008 triunfo dr
30020 triunfo dr
29874 triunfo dr
29895 triunfo dr
29866 triunfo dr
29886 triunfo dr
29936 triunfo dr
30015 triunfo dr
29852 triunfo dr
30012 triunfo dr
29930 triunfo dr
30035 triunfo dr
30055 triunfo dr
30025 triunfo dr
30066 triunfo dr
29932 triunfo dr
30060 triunfo dr
29901 triunfo dr
2337 troutdale dr
2363 troutdale dr
2468 troutdale dr
2220 careful ave
2234 careful ave
2228 careful ave
2230 careful ave
2226 careful ave

29201 larkspur ln
29200 larkspur ln
6375 zuma mesa dr
6375 zuma mesa dr
6271 zuma mesa dr
6414 zuma mesa dr
6395 zuma mesa dr
6385 zuma mesa dr
6277 zuma mesa dr
6300 zuma mesa dr
28903 wight rd
28901 wight rd
28908 wight rd
28908 wight rd
28904 wight rd
28929 wight rd
28933 wight rd
28911 wight rd
28912 wight rd
28517 pacific coast
28920 pacific coast
27841 pacific coast
29590 heathercliff rd
29590 heathercliff rd
29590 heathercliff rd
29590 heathercliff rd
29590 heathercliff rd
29590 heathercliff rd
29590 heathercliff rd
29590 heathercliff rd
29590 heathercliff rd
29590 heathercliff rd
29590 heathercliff rd
29590 heathercliff rd
29590 heathercliff rd
29590 heathercliff rd
29590 heathercliff rd
29590 heathercliff rd
29590 heathercliff rd
29590 heathercliff rd
29590 heathercliff rd
29590 heathercliff rd
29590 heathercliff rd
29590 heathercliff rd
29590 heathercliff rd
29590 heathercliff rd
29590 heathercliff rd
29590 heathercliff rd
29590 heathercliff rd
29590 heathercliff rd
29590 hea

6400 trancas canyon rd
5867 trancas canyon rd
6432 trancas canyon rd
6364 trancas canyon rd
6120 trancas canyon rd
5981 trancas canyon rd
5999 trancas canyon rd
31222 beach view estates dr
31250 beach view estates dr
31255 beach view estates dr
31209 beach view estates dr
31220 beach view estates dr
31220 beach view estates dr
31011 bailard rd
31113 bailard rd
31021 bailard dr
31131 bailard dr
31021 bailard rd
31241 bailard rd
31223 bailard rd
31213 bailard rd
31217 bailard rd
31275 bailard rd
31201 bailard rd
31243 bailard rd
31237 bailard rd
31273 bailard rd
31215 bailard rd
31121 bailard rd
5924 paseo canyon rd
5958 paseo canyon dr
6015 paseo canyon dr
6025 paseo canyon dr
6036 paseo canyon rd
5936 paseo canyon dr
6035 paseo canyon dr
6071 paseo canyon dr
6053 paseo canyon dr
6079 paseo canyon dr
6328 paseo canyon dr
6316 paseo canyon rd
6325 paseo canyon dr
5942 paseo canyon dr
5975 paseo canyon dr
5964 paseo canyon dr
6307 paseo canyon dr
6334 paseo canyon dr
6115 paseo canyon dr


27420 winding way
27929 winding way
27856 winding way
27434 winding way
27860 winding way
27454 winding way
27935 winding way
27767 winding way
27475 winding way
28099 paquet place
28099 paquet place
28011 paquet pl
28935 west beach ln
6360 delaplane rd
6420 delaplane rd
6380 delaplane rd
6208 delaplane rd
6350 delaplane rd
6401 zumirez dr
6401 zumirez dr
6556 zumirez dr
6665 zumirez dr
6685 zumirez dr
6625 zumirez dr
6649 zumirez dr
5864 zumirez dr
6570 zumirez dr
6655 zumirez dr
6540 zumirez dr
6551 zumirez dr
6601 zumirez dr
6565 zumirez dr
6329 zumirez dr
6604 zumirez dr
5908 zumirez dr
5810 zumirez dr
6363 zumirez dr
6637 zumirez dr
6531 zumirez dr
5815 zumirez dr
6613 zumirez dr
6656 zumirez dr
6648 zumirez dr
6309 zumirez dr
6647 zumirez dr
6003 zumirez dr
5820 zumirez dr
6324 zumirez dr
5868 zumirez dr
6611 prtshead rd
6611 prtshead rd
28901 selfridge dr
28935 selfridge dr
28869 selfridge dr
28871 selfridge dr
28873 selfridge dr
28843 selfridge rd
28863 selfridge dr
28863 selfr

28605 wagon rd
26119 kenrose cir
26147 kenrose cir
26125 kenrose cir
26144 kenrose cir
26143 kenrose cir
26130 kenrose cir
26152 kenrose cir
26124 kenrose cir
26135 kenrose cir
26136 kenrose cir
26115 kenrose cir
26148 kenrose cir
5389 fremantle ln
5309 fremantle ln
5405 fremantle ln
5375 fremantle ln
5331 fremantle ln
5321 fremantle ln
129 fallen leaf ln
128 fallen leaf ln
131 fallen leaf ln
125 fallen leaf ln
132 fallen leaf ln
130 fallen leaf ln
127 fallen leaf ln
126 fallen leaf ln
26124 roymor dr
26114 roymor dr
26115 roymor dr
26111 roymor dr
26121 roymor dr
26118 roymor dr
26130 roymor dr
26134 roymor dr
26103 roymor dr
6485 zuma view pl
6465 zuma view pl
6473 zuma view pl
6471 zuma view pl
6453 zuma view pl
6481 zuma view pl
6475 zuma view pl
6487 zuma view pl
6467 zuma view pl
6543 zuma view pl
6535 zuma view pl
6463 zuma view pl
6451 zuma view pl
6483 zuma view pl
6479 zuma view pl
5334 ambridge dr
5315 ambridge dr
5356 ambridge dr
5338 ambridge dr
5345 ambridge dr
5333 ambri

In [71]:
newdf = pd.DataFrame(new_list)

In [72]:
len(newdf.loc[newdf.score>0])

2968

In [75]:
newdf.loc[newdf.score>95]

,infile_address,match_address,score
0,33360 decker school rd,33360 decker school rd,100
3,33257 decker school rd,33257 decker school rd,100
8,33165 decker school rd,33165 decker school rd,100
9,33169 decker school rd,33169 decker school rd,100
10,33169 decker school rd,33169 decker school rd,100
...,...,...,...
3729,6907 grasswood ave,6907 grasswood ave,100
3731,2770 searidge st,2770 searidge st,100
3733,6809 seawatch ln,6809 seawatch ln,100
3737,6801 las olas way,6801 las olas way,100


In [76]:
newdf.to_csv('alladdress_matches.csv')

In [77]:
newest = pd.read_csv(r'C:\Users\i23733\Desktop\notebooks\alladdress_matches.csv')

In [114]:
newest['Address1'] = newest.infile_address.astype(str)

In [116]:
woolsey.dtypes

fid               int64
Address          object
CITY             object
Address1         object
PartialStreet    object
dtype: object

In [117]:
woolseymatch = woolsey.join(newest, on='Address1')

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [101]:
len(woolseymatch)

44058

In [102]:
woolseymatch.head()

,fid,Address,CITY,Address1,PartialStreet,infile_address,match_address,score,match_big
0,1,33360 Decker School Rd,Malibu,33360 Decker School Rd,33360 Decker School,33360 Decker School Rd,33360 Decker School Rd,100,33360 Decker School Rd
1,3,415 Westlake Blvd,Malibu,415 Westlake Blvd,415 Westlake,415 Westlake Blvd,415 Westlake Blvd,100,415 Westlake Blvd
2,5,33457 Pacific Coast Hwy,Malibu,33457 Pacific Coast Hwy,33457 Pacific Coast,33457 Pacific Coast Hwy,33457 Pacific Coast Hwy,100,33457 Pacific Coast Hwy
3,6,31839 Lobo Canyon Rd,Agoura Hills,31839 Lobo Canyon Rd,31839 Lobo Canyon,31839 Lobo Canyon Rd,31839 Lobo Canyon Rd,100,31839 Lobo Canyon Rd
4,11,5900 Bonsall Dr,Malibu,5900 Bonsall Dr,5900 Bonsall,5900 Bonsall Dr,5900 Bonsall Dr,100,5900 Bonsall Dr
